## Create USGS Gage Shapefile

This notebook provides a method to create a shapefile for a USGS gage using provided metadata and a given projection system.

In [31]:
# Import necessary libraries
import os
import geopandas as gpd
from shapely.geometry import Point, mapping
import fiona
from fiona.crs import from_string

### Function to Create the Shapefile

In [32]:
def create_usgs_gage_shapefile_with_fiona(prj_file_path, output_shapefile_path):
    # Read the projection system from the provided .prj file
    with open(prj_file_path, 'r') as file:
        projection_system = file.read()

    # Define the latitude and longitude
    latitude = 29.67555556
    longitude = -94.6661139

    # Create a GeoDataFrame using the provided decimal latitude and longitude
    # Initially, we'll use a basic WGS84 CRS
    geometry = [Point(longitude, latitude)]
    df = gpd.GeoDataFrame(
        {
            'Agency': ['U.S. Geological Survey'],
            'Site_ID': ['08042558'],
            'Site_name': ['W Fk Double Bayou at Eagle Ferry Rd nr Anahuac, TX'],
            'Site_type': ['Stream'],
            'District': ['Texas'],
            'State': ['Texas'],
            'County': ['Chambers County'],
            'Country': ['US'],
            'Subbasin': ['12040202'],
            'Date_site': ['20060509'],
            'Drain_area': [22.2],
            'Contrib_DA': [22.2],
            'ProjectNo': ['8653-9VR11'],
            'Latitude': [latitude],
            'Longitude': [longitude]
        },
        geometry=geometry,
        crs="EPSG:4326"  # WGS84 CRS
    )

    # Reproject the GeoDataFrame to the CRS from the .prj file
    df = df.to_crs(projection_system)

    # Define the metadata
    meta = {
        'crs': from_string(projection_system),
        'schema': {
            'geometry': 'Point',
            'properties': {col: 'str' for col in df.columns if col != 'geometry'}
        },
        'driver': 'ESRI Shapefile'
    }

    # Use fiona to write the shapefile
    with fiona.open(output_shapefile_path, 'w', **meta) as sink:
        for _, row in df.iterrows():
            sink.write({
                'geometry': mapping(row['geometry']),
                'properties': {col: row[col] for col in df.columns if col != 'geometry'}
            })


### Usage

In [33]:

# Provide the path to your .prj file
prj_file_path = r"Z:\\GLO\\GLO_Task\\Gage_Shapefile\\NAD_1983_TCMS_Albers_FtUS.prj"

# Set the output path for the shapefile in the same directory as the .prj file
output_directory = os.path.dirname(prj_file_path)
output_shapefile_path = os.path.join(output_directory, "USGS_gage_08042558.shp")

create_usgs_gage_shapefile_with_fiona(prj_file_path, output_shapefile_path)


C:\Users\asaharia\AppData\Local\Temp\ipykernel_6848\1898180662.py:8: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_string() instead.
  create_usgs_gage_shapefile_with_fiona(prj_file_path, output_shapefile_path)
